In [10]:
# ultra_simple_solution.py
import os
# SOLUTION ULTIME - Forcer PyTorch seulement
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["USE_TORCH"] = "1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Désactiver tous les imports TensorFlow
os.environ["NO_TF"] = "1"
os.environ["NO_KERAS"] = "1"

print("🔧 Configuration de l'environnement...")

# Importer dans l'ordre correct
import torch
print(f"PyTorch: {torch.__version__}")

from datasets import load_dataset
print("✅ datasets importé")

from transformers import AutoTokenizer, AutoModelForCausalLM
print("✅ transformers importé")

from peft import LoraConfig, get_peft_model
print("✅ peft importé")

# Éviter d'importer Trainer si problème
from transformers import TrainingArguments

print("🎉 Tous les imports réussis!")
# ... reste du code identique


# 1 Choisir ton modèle de base
base_model = "distilgpt2"  # ou "meta-llama/Llama-2-7b-chat-hf" si tu as la VRAM
tokenizer = AutoTokenizer.from_pretrained(base_model)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 2 Charger ton dataset JSON
dataset = load_dataset("json", data_files="data/analyzer_dataset.json")
train_data = dataset["train"]

def tokenize_fn(example):
    prompt = f"Context: {example['context']}\nInsight:"
    target = example["insight"]
    full = prompt + target
    tokens = tokenizer(full, truncation=True, padding="max_length", max_length=256)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

train_data = train_data.map(tokenize_fn, batched=True, remove_columns=train_data.column_names)

# 3 Charger le modèle et ajouter LoRA
model = AutoModelForCausalLM.from_pretrained(base_model)
lora_cfg = LoraConfig(
    r=8,                    # rank low-dim
    lora_alpha=32,          # scaling
    target_modules=["c_attn", "q_proj", "v_proj"],  # selon modèle
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_cfg)

# 4  Entraînement
training_args = TrainingArguments(
    output_dir="./lora_analyzer",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=50,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    train_dataset=train_data,
    args=training_args,
    tokenizer=tokenizer
)

trainer.train()

# 5 Sauvegarder les poids LoRA
model.save_pretrained("./lora_analyzer")


🔧 Configuration de l'environnement...
PyTorch: 2.7.0+cpu
✅ datasets importé
✅ transformers importé
✅ peft importé
🎉 Tous les imports réussis!
✅ peft importé
🎉 Tous les imports réussis!


FileNotFoundError: Unable to find 'C:/Users/user/OneDrive/Desktop/LLM_RL/Multi-Agent-LLms-with-Reinforcement-Learning/train\data/analyzer_dataset.json'

In [ ]:
# Test du modèle fine-tuné
from peft import PeftModel, PeftConfig

# Charger le modèle de base
base_model = AutoModelForCausalLM.from_pretrained("distilgpt2")
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

# Charger les poids LoRA
config = PeftConfig.from_pretrained("./lora_analyzer")
model = PeftModel.from_pretrained(base_model, "./lora_analyzer")
model.to(device)

# Test avec un nouvel exemple
test_input = "Context: state: backlog=high, team_size=reduced\nInsight:"
inputs = tokenizer(test_input, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=0.7,
        num_return_sequences=1
    )
    
generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\nTest generation:")
print("Input:", test_input)
print("Generated insight:", generated[len(test_input):].strip())